Competition page: https://www.kaggle.com/c/vietai-foundation-course-cnn-assignment  
Original dataset: https://www.kaggle.com/c/vietai-foundation-course-cnn-assignment/data  
Extracted dataset (to be used in this notebook): https://www.kaggle.com/aeryss/vietai-c6-assignment3-extracted-dataset  

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import tensorflow.keras as tfk
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, BatchNormalization, MaxPool2D, GlobalAveragePooling2D, Dense, Dropout, Activation
import pandas as pd
import numpy as np

In [ ]:
# # Data is seperated using this piece of code:

# import shutil
# train_dir = "Assignment3Data/train"
# test_dir = "Assignment3Data/test"


# try:
#     shutil.rmtree(train_dir)
# except Exception:
#     pass
# try:
#     shutil.rmtree(test_dir)
# except Exception:
#     pass
    
# os.makedirs(train_dir)
# os.makedirs(test_dir)

# train_y = train_df.label
# num_classes = len(np.unique(train_y))
# y_ohe = tf.keras.utils.to_categorical(train_y, num_classes=num_classes)

# classes = ["book", "can", "cardboard", "glass_bottle", "pen", "plastic_bottle"]
# for eachClass in classes:
#     os.makedirs(train_dir + "/" + eachClass)

# train_file_path = [os.path.join("../input/vietai-foundation-course-cnn-assignment/Assignment3Data/images", file) for file in train_df.image]
# test_file_path = [os.path.join("../input/vietai-foundation-course-cnn-assignment/Assignment3Data/images", file) for file in test_df.image]
# # for train_image in train_file_path:
# #     shutil.copy(train_image, train_dir)

# for i in range(len(train_file_path)):
#     shutil.copy(train_file_path[i], train_dir + "/" + classes[train_df.loc[i, "label"]])
# for test_image in test_file_path:
#     shutil.copy(test_image, test_dir)
    


In [ ]:
data_dir = "../input/vietai-c6-assignment3-extracted-dataset/train.csv"
sub_dir = "../input/vietai-c6-assignment3-extracted-dataset/sample_submission.csv"
train_df = pd.read_csv(data_dir)
sub_df = pd.read_csv(sub_dir)

In [ ]:
train_df.head()

In [ ]:
sub_df.head()

In [ ]:
classes = ["book", "can", "cardboard", "glass_bottle", "pen", "plastic_bottle"]
train_y = train_y = train_df.label
num_classes = len(np.unique(train_y))
y_ohe = tf.keras.utils.to_categorical(train_y, num_classes=num_classes)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

size = 224
batch_size=32
# Use ImageDataGenerator, because converting into NumPy is costly
train_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2) # Intended to aug images, but it is costly
train_gen = train_data_gen.flow_from_directory("../input/vietai-c6-assignment3-extracted-dataset/train", batch_size=batch_size,
                                              target_size=(size, size), subset="training")
valid_gen = train_data_gen.flow_from_directory("../input/vietai-c6-assignment3-extracted-dataset/train", batch_size=batch_size,
                                              target_size=(size, size), subset="validation")

In [ ]:
# # Tried using TPU but always fail, don't know why


# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# Basic model
def create_model():
    model = tfk.Sequential()
    model.add(Conv2D(144, (3, 3), strides=(1, 1), padding="valid", input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Conv2D(94, (3, 3), strides=(1, 1), padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Conv2D(144, (3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Conv2D(94, (3, 3), strides=(1, 1), padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Conv2D(42, (3, 3), strides=(1, 1), padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dropout(0.21))
    model.add(Dense(256, activation="tanh", kernel_regularizer=tf.keras.regularizers.l2()))
    model.add(Dropout(0.21))
    model.add(Dense(256, activation="tanh", kernel_regularizer=tf.keras.regularizers.l2()))
    model.add(Dropout(0.21))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    print(model.summary())
    return model

In [ ]:
# Pretrained model, best with DenseNet, 
# Xception, NASNet, ResNet, EfficientNet, all were failed although the Dense layers were the same
def create_pretrained_model():
    model = tfk.Sequential()
    pretrained_net = tfk.applications.DenseNet201(
        include_top=False,
        input_shape=(224, 224, 3),
        pooling="avg"
    )
    model.add(pretrained_net)
    model.add(Dropout(0.42))
    model.add(Dense(256, activation="tanh", kernel_regularizer=tf.keras.regularizers.l2()))
    model.add(Dropout(0.21))
    model.add(Dense(128, activation="tanh"))
    model.add(Dropout(0.21))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    print(model.summary())
    return model

In [ ]:
# Callbacks when val_acc reaches 99%, but best was 96% unfortunately
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get("val_accuracy") is not None:
            if(logs.get('val_accuracy') > 0.99):
                print("\nReached 99% val_accuracy so cancelling training!")
                self.model.stop_training = True
mcb = myCallback()

In [ ]:
epochs = 150
use_efficientnet = True
if not use_efficientnet:
    model = create_model()
else:
    model = create_pretrained_model()


# Learning rate decay of pretrained model, taken from the notebook TPUs in Colab
start_lr = 0.00001
min_lr = 0.00001
max_lr = 0.00005*8
rampup_epochs = 5
sustain_epochs = 0
exp_decay = .8

def lrfn(epoch):
    if epoch < rampup_epochs:
        return (max_lr - start_lr)/rampup_epochs * epoch + start_lr
    elif epoch < rampup_epochs + sustain_epochs:
        return max_lr
    else:
        return (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr

mcp = tf.keras.callbacks.ModelCheckpoint("my_model.h5", monitor="val_accuracy",
                        save_best_only=True, save_weights_only=True) # Save weights of best model based on val_acc
val_acc_earlyStop = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", 
                                                    patience = epochs//15, restore_best_weights = True) # If val_acc does not improve
lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=True)

model.fit(train_gen, validation_data=valid_gen, epochs=epochs, callbacks=[mcp, mcb, val_acc_earlyStop, lr_callback])

In [ ]:
model = create_pretrained_model()
model.build(input_shape=(1,224,224,3))
model.load_weights("my_model.h5")

In [ ]:
test_data_gen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_data_gen.flow_from_directory("../input/vietai-c6-assignment3-extracted-dataset/test",class_mode=None, target_size=(size, size), shuffle=False)
test_generator.reset()
pred = model.predict(test_generator)

# Get best prediction
pred_labels = np.argmax(pred, axis=1)
sub_df['label'] = pred_labels
sub_df.head(20)

In [ ]:
sub_df.to_csv("submission.csv", index=False)